In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import DataPreprocessor, get_load_config_from_yaml
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.keras_models as Models
import core
import os

# Configuration
CONFIG_PATH = "../config/test_load_config.yaml"


# Set plotting style
plt.rcParams.update({"font.size": 18})

# Load data
load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)

data_config = DataProcessor.load_from_npz(
    load_config.data_path, event_numbers="odd", max_events=2_000_000
)
X, y = DataProcessor.get_data()
del DataProcessor

## Load Model and Setup Evaluator

In [ ]:
import core.reconstruction as BaselineMethods
import core.evaluation.reconstruction_evaluator as Evaluation
from core.reconstruction import (
    GroundTruthReconstructor,
    PerfectAssignmentReconstructor,
)

reload(Evaluation)
reload(BaselineMethods)
ground_truth_assigner = GroundTruthReconstructor(data_config, use_nu_flows=True, assignment_name="")
transformer = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"Transformer (PtEtaPhi) $\nu$", assignment_name=""
)
MODEL_DIR = "../models/binned_regression/"
transformer.load_model(f"{MODEL_DIR}/odd_model.keras")


prediction_manager = Evaluation.PredictionManager(
    [ground_truth_assigner, transformer],
    X,
    y,
)

evaluator = Evaluation.ReconstructionPlotter(prediction_manager)

In [ ]:
transformer.model.summary()

In [ ]:
PLOTS_DIR = "plots/Binned_regression_transformer/"
os.makedirs(PLOTS_DIR, exist_ok=True)

In [ ]:
evaluator.save_regression_error_latex_table(save_dir=f"{PLOTS_DIR}", n_bootstrap=10)

In [ ]:
np.max(prediction_manager.get_neutrino_predictions(1), axis=0)

In [ ]:
fig, ax = evaluator.plot_relative_neutrino_deviations(
    bins=30,
    coords="spherical_lepton_fixed",
    xlims=[(-1, 8), (0, 3), (-100 / 1e3, 100 / 1e3)],
)
fig.savefig(
    f"{PLOTS_DIR}/neutrino_reconstruction_comparison_spherical_lepton.pdf",
    bbox_inches="tight",
)

In [ ]:
fig, ax = evaluator.plot_relative_neutrino_deviations(bins=30, coords="cartesian")
fig.savefig(f"{PLOTS_DIR}/neutrino_reconstruction_comparison.pdf")

In [ ]:
fig, ax = evaluator.plot_relative_neutrino_deviations(bins=30, coords="spherical")
fig.savefig(
    f"{PLOTS_DIR}/neutrino_reconstruction_comparison_spherical.pdf", bbox_inches="tight"
)